In [1]:
import os
from glob import glob
import re
import tqdm
import unicodedata

In [4]:
filepath = 'out/whisper-large-v2-100-output-test/Xóm Đêm.json'
import json
with open(filepath, 'r') as f:
    file = json.load(f)  

In [ ]:
import json

# Load the JSON file with Unicode escape sequences
input_file = "out/whisper-large-v2-100-output-test/Xóm Đêm.json"  # Replace with your file path
output_file = "out/whisper-large-v2-100-output-test/Xóm Đêm_utf.json"  # Output path for the converted file

import glob 

input_files = glob.glob("out/whisper-large-v2-100-output-test/*.json")
for input_file in input_files:
    with open(input_file, "r", encoding="utf-8") as file:
        data = json.load(file)  # This loads and decodes the JSON

    output_file = input_file.replace('whisper-large-v2-100-output-test', 'whisper-large-v2-100-output-test-utf')
    # Re-save the JSON without Unicode escape sequences
    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)  # `ensure_ascii=False` avoids escape sequences


In [3]:
def normalize_VNmese(text):
    return unicodedata.normalize('NFC', text)

In [5]:
output_data_path = 'out/master_1st_100_preprocessed'
os.makedirs(output_data_path, exist_ok=True)

*Processing Lyrics*

In [6]:
data_path = 'out/master_1st_100'
lyric_files = glob(data_path + '/*.origin.lrc')
song_files = glob(data_path + '/*.mp3')
print(len(lyric_files))

lyric_files = [normalize_VNmese(x) for x in lyric_files]
song_files = [normalize_VNmese(x) for x in song_files]

100


In [7]:
'''
Remove the prefix if it has the line about Ca sĩ: 
[00:00.00]Bài hát: 4 Phút 20 Giây
[00:02.00]Ca sĩ: Hà Anh
[00:04.00]
'''
IDENTIFIER = ']ca sĩ:'
def remove_prefix(lyrics):

    if IDENTIFIER in lyrics.lower():
        lyrics = lyrics.split('\n')
        id_sent = [x for x in lyrics if IDENTIFIER in x.lower()][0]
        
        lyrics = lyrics[lyrics.index(id_sent)+1:]
        lyrics = '\n'.join(lyrics)
    return lyrics

'''
Remove all timestamps without lyrics
'''
def remove_silence(lyrics):
    lyrics = lyrics.strip().split('\n')
    lyrics = [x.strip() for x in lyrics]

    for idx, line in enumerate(lyrics):
        if line == '':
            continue
        if line[0]== '[' and line[-1] == ']':
            lyrics[idx] = ''

    lyrics = [x.strip() for x in lyrics if x != '']
    return '\n'.join(lyrics)

'''
Remove .... and redundant spaces
'''

def replace_punc(text):
    lines = text.strip().split('\n')
    for i in range(len(lines)):
        # Replace three or more dots with a single space
        lines[i] = re.sub(r'\.{2,}', ' ', lines[i])
        # Replace two or more spaces with a single space
        lines[i] = re.sub(r'\s{2,}', ' ', lines[i])
    return '\n'.join(lines)



In [8]:
with_prefix = 0
for lyric_file in tqdm.tqdm(lyric_files):
    
    with open(lyric_file, 'r', encoding='utf-8-sig') as f:
        text = f.read()
    text = normalize_VNmese(text)

 
    text = remove_prefix(lyrics=text)


    text = replace_punc(text)  
          
    text = remove_silence(text)

    
    with open(os.path.join(output_data_path, os.path.basename(lyric_file)), 'w') as f:
        f.write(text)


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 727.53it/s]


*PROCESSING SONGS*

In [9]:
import librosa
import soundfile as sf

'''
Cut the audio file 5 seconds after the last timestamp
'''

def cut_postfix(audio_path,lyric_path, output_folder, extra_seconds=10):
    #GET AUDIO
    audio, sr = librosa.load(audio_path, sr=None)  # sr=None keeps the original sample rate
    # Calculate the duration manually
    song_duration = len(audio) / sr

    #GET LYRICS
    with open(lyric_path, 'r', encoding='utf-8-sig') as f:
        text = f.readlines()
    last_line = text[-1]
    # Regular expression to extract content inside square brackets
    match = re.search(r'\[(.*?)\]', last_line)

    # Check if a match is found
    if match:
        time_str = match.group(1)  # Extract the content inside brackets
    minute, seconds = [float(x) for x in time_str.split(':')]
    lyrics_duration = minute*60 + seconds

    if song_duration - lyrics_duration > 15:
        print(audio_path)
        print(song_duration, lyrics_duration)
        # Cut off from lyric duration + 10 seconds
        cutoff_sample = int((lyrics_duration+extra_seconds) * sr)
        audio = audio[:cutoff_sample]
    
    sf.write(os.path.join(output_folder, os.path.basename(audio_path)), audio, sr)

In [10]:
for audio_file in tqdm.tqdm(song_files):
    songname = os.path.basename(audio_file).split('.mp3')[0]
    print(songname)
    lyric_file = os.path.join(output_data_path, songname + '.origin.lrc')
    cut_postfix(audio_path=audio_file,lyric_path=lyric_file, output_folder=output_data_path)


  0%|          | 0/100 [00:00<?, ?it/s]

Em LÀ Ánh Dương
out/master_1st_100/Em LÀ Ánh Dương.mp3
247.11836734693878 220.11


  1%|          | 1/100 [00:34<56:29, 34.24s/it]

Tình Ca Mùa Xuân
out/master_1st_100/Tình Ca Mùa Xuân.mp3
273.1619727891156 251.8


  2%|▏         | 2/100 [00:45<33:35, 20.56s/it]

...

 98%|█████████▊| 98/100 [16:16<00:17,  8.87s/it]

Anh Không Làm Được
out/master_1st_100/Anh Không Làm Được.mp3
311.79755102040815 284.63


 99%|█████████▉| 99/100 [16:26<00:09,  9.23s/it]

Cả Một Đời Để Quên (Meme Lofi)


100%|██████████| 100/100 [16:38<00:00,  9.98s/it]
